In [ ]:
%load_ext autoreload
%autoreload 2

# Generation

> Efficient generation with flexible stopping criteria


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
from dart_math.gen import *

## Difficulty-Aware Rejection Sampling in 5 Lines of Code


```python
from dart_math.data import load_query_dps
from dart_math.gen import gen, is_dp_dars_finished
from dart_math.eval import EvaluatorMathBatch
# ...
generator = Generator(llm, sampling_params, resp_sample_cls=RespSampleVLLM, batch_evaluator=(EvaluatorMathBatch() if not args.gen_only else None), code_exec_cfg=CodeExecCfg.load_from_id_or_path(args.code_exec_cfg) if args.code_exec_cfg else None)
generator.gen(query_dps=query_dps, dp_stop_criteria=is_dp_dars_finished, save_path=args.gen_save_path, n_paths_per_save=args.save_gen_path_bs)
```

1. `generator.gen` generates with the vLLM model `llm` using sampling parameters `sampling_params` on query data points `query_dps` until every data point meets the stopping criteria `dp_stop_criteria`.
2. Samples are generated in batch and evaluated with `batch_evaluator` if specified.
3. Generated samples are saved to `save_path`.

For a more detailed usage example, please refer to our [generation script for DART-Math](https://github.com/hkust-nlp/dart-math/blob/main/pipeline/gen.py).


In [ ]:
show_doc(Generator, title_level=3)

In [ ]:
show_doc(Generator.gen, title_level=4)

In [ ]:
show_doc(Generator.gen_pure, title_level=4)

## API Reference


### Data Preprocessing


In [ ]:
show_doc(get_icl_egs, title_level=4)

In [ ]:
show_doc(get_prompt_template4model, title_level=4)

### Stopping Criteria


In [ ]:
show_doc(is_dp_dars_finished, title_level=4)

### IO


In [ ]:
show_doc(get_res_fname, title_level=4)